# Dependencies

In [1]:

import plotly.express as px
from jupyter_dash import JupyterDash
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np

def read_data(name = 'btcusd'):
    import pandas as pd
    path = f"data/{name}.csv"
    df =pd.read_csv(path)    
    return df



## graph_indicators function

In [2]:
 # to use this function: run this block
# avaliable time_frame inputs: ['24H', '1W','1M','6M','1Y','ALL']
def graph_indicators(data, time_frame = 'ALL',ema_periods = 10, bollinger_rate = 20 ):
    import pandas as pd
    import numpy as np
    import plotly.express as px
    from datetime import datetime, date
    import plotly.graph_objects as go
    

    # This function converts the timestamp into pandas datetime and set it as index
    def change_date(df):
        x = [i for i in range(len(df.columns.isin(['time','open','close','high','low','volume'])-1)) if df.columns.isin(['time','open','close','high','low','volume'])[i] == False ]
        df.drop(columns=df.columns[x],inplace =True)
        df['time']= pd.to_datetime(df['time'],unit = 'ms')
        df = df.set_index('time',drop = True)
        df.sort_index
        return df

    # this function calculates the ema
    def calculate_ema(prices, days = 10, smoothing=2):
        ema = [0]*(days-1) +[sum(prices[:days]) / days] # First method
        for price in prices[days:]:
            ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
        return ema

    # this function resamples the data into sepcified resolution
    def resample(data,resolution ='D'):
        data = data.resample(resolution).mean()
        data = data.dropna()
        return data

    #calculating the simple moving average
    def get_sma(prices, rate= bollinger_rate): 
        return prices.rolling(rate).mean()

    # Calculate the bullinger bands (Both upper & lower bonds)
    def get_bollinger_bands(prices, rate=bollinger_rate):
        sma = get_sma(prices, rate)
        two_std = 2*prices.rolling(rate).std()
        bollinger_top = sma + two_std # Calculate top band
        bollinger_below = sma - two_std # Calculate bottom band
        return bollinger_top, bollinger_below

    # this function will return a DataFrame with specified parameters,
    # later the data produced will be used to graph
    # The "resolution" parameter will be determined based on "time_frame"
    def process_data(data,resolution = 'D', ema_periods = 10, bollinger_rate = 20):
        # first convert the data
        data = change_date(data)

        #then resample the data
        if resolution != 'Min':
            data = resample(data,resolution)

        # then calculate and stores the EMA
        data['ema'] = calculate_ema(data.close, days= ema_periods )

        # Calculate and store the SMA
        data['sma'] = get_sma(data.close, rate = bollinger_rate)

        # calculate the upper & lower bollinger bands and store it
        bollinger_up, bollinger_down = get_bollinger_bands(prices = data.close,rate = bollinger_rate)
        data['bollinger_up'] = bollinger_up
        data['bollinger_down'] = bollinger_down

        # return the DataFrame with all data processed, and Includes all Tech Indicators calculated based on
        # specified resolution 
        return data
    
    def make_graphs(data, columns = ['close']):
        import pandas as pd
        import plotly.express as px
        import plotly.graph_objects as go
        
        figure = px.line(data_frame = data, y =columns)
        layout = go.Layout(
            plot_bgcolor='#f5f5f5',
            # Font Families
            font_family='Balto',
            font_color='#000000',
            font_size=18,
            xaxis=dict(
                rangeslider=dict(
                    visible=False
                )
            )
        )
        # Update options and show plot
        figure.update_layout(layout)
        return figure



    # dealing with Different Timeframe, assign a resolution to each time frame
    # outputs a plotly express plot

    if time_frame == '24H':
        # the number inside tail() is (how many min in 24h) so --> (number of 'resolution' in 'time_frame')
        graph_data = process_data(data,resolution='min',ema_periods=ema_periods,bollinger_rate=bollinger_rate).tail(1440)
        figure = make_graphs(data= graph_data, columns =['close','ema','bollinger_up','bollinger_down'])
        return figure

    if time_frame == 'ALL':
        graph_data = process_data(data,resolution='D',ema_periods=ema_periods,bollinger_rate=bollinger_rate)
        figure = make_graphs(data= graph_data, columns =['close','ema','bollinger_up','bollinger_down'])
        return figure

    if time_frame == '1W':
        graph_data = process_data(data,resolution='15Min',ema_periods=ema_periods,bollinger_rate=bollinger_rate).tail(672)
        figure = make_graphs(data= graph_data, columns =['close','ema','bollinger_up','bollinger_down'])
        return figure

    if time_frame == '1M':
        graph_data = process_data(data,resolution='3H',ema_periods=ema_periods,bollinger_rate=bollinger_rate).tail(240)
        figure = make_graphs(data= graph_data, columns =['close','ema','bollinger_up','bollinger_down']) 
        return figure

    if time_frame == '6M':
        graph_data = process_data(data,resolution='D',ema_periods=ema_periods,bollinger_rate=bollinger_rate).tail(180)
        figure = make_graphs(data= graph_data, columns =['close','ema','bollinger_up','bollinger_down'])
        return figure

    if time_frame == '1Y':
        graph_data = process_data(data,resolution='D',ema_periods=ema_periods,bollinger_rate=bollinger_rate).tail(365)
        figure = make_graphs(data= graph_data, columns =['close','ema','bollinger_up','bollinger_down'])
        return figure       

    # if something not included gets put iin, just graph the all time data 
        # without any indicators
    else:
        graph_data = process_data(data = data, resolution = 'D')
        figure = make_graphs(data = graph_data, columns =['close'])
        return figure    







## graph_macd function

In [3]:


# this is the dedicated function for MACD analysis
# TO USE THE FUNCTION, RUN THIS BLOCK
def graph_macd(data,time_frame = 'all', macd_min = 12, macd_max = 26 ):
    import pandas as pd
    import numpy as np
    import plotly.express as px
    from datetime import datetime, date
    import plotly.graph_objects as go

 
    

    # This function converts the timestamp into pandas datetime and set it as index
    def change_date(df):
        x = [i for i in range(len(df.columns.isin(['time','open','close','high','low','volume'])-1)) if df.columns.isin(['time','open','close','high','low','volume'])[i] == False ]
        df.drop(columns=df.columns[x],inplace =True)
        df['time']= pd.to_datetime(df['time'],unit = 'ms')
        df = df.set_index('time',drop = True)
        df.sort_index
        return df


    # this function resamples the data into sepcified resolution
    def resample(data,resolution ='D'):
        data = data.resample(resolution).mean()
        data = data.dropna()
        return data


    def get_macd(prices,min_periods = macd_min, max_periods = macd_max):
        # Get the 26-day EMA of the closing price
        k = prices.ewm(span=12, adjust=False, min_periods=min_periods).mean()
        # Get the 12-day EMA of the closing price
        d = prices.ewm(span=26, adjust=False, min_periods=max_periods).mean()
        #Subtract the 26-day EMA from the 12-Day EMA to get the MACD
        macd = k - d
        # Get the 9-Day EMA of the MACD for the Trigger line
        macd_signal = macd.ewm(span=9, adjust=False, min_periods=9).mean()
        # Calculate the difference between the MACD - Trigger for the Convergence/Divergence value
        macd_CD = macd - macd_signal 
        return macd, macd_signal, macd_CD


    # this function will return a DataFrame with specified parameters,
    # later the data produced will be used to graph
    # The "resolution" parameter will be determined based on "time_frame"
    def process_data(data,resolution = 'D', macd_min = 12, macd_max= 26):
        # first convert the data
        data = change_date(data)

        #then resample the data
        # if the resolution is not already "Min"
        if resolution != 'Min':
            data = resample(data,resolution)

        # calculate the MACD & MACD signal
        macd, macd_signal, macd_CD = get_macd(data.close, min_periods=macd_min, max_periods= macd_max)
        data['macd'] = macd
        data['macd_signal'] = macd_signal
        data['macd_CD'] = macd_CD


        # return the DataFrame with all data processed, and Includes all Tech Indicators calculated based on
        # specified resolution 
        return data



    def make_graphs(data):
        import numpy as np
        import plotly.express as px
        from datetime import datetime, date
        import plotly.graph_objects as go
        from plotly.subplots import make_subplots
        # Construct a 2 x 1 Plotly figure

        fig = make_subplots(rows=2, cols=1)
        # price Line
        fig.append_trace(go.Scatter(
        x=data.index,
        y=data['close'],
        line=dict(color='#6495ed', width=1),
        name='closing price',
        # showlegend=False,
        legendgroup='1',
    ), row=1, col=1)
        # Candlestick chart for pricing
        fig.append_trace(
            go.Candlestick(
                x=data.index,
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close'],
                increasing_line_color= 'green',#'#ffa500',
                decreasing_line_color='red',
                showlegend=False
            ), row=1, col=1
        )
        # Fast Signal (%k)
        fig.append_trace(
            go.Scatter(
                x=data.index,
                y=data['macd'],
                line=dict(color='#ff7f50', width=2),
                name='MACD',
                #showlegend=False,
                legendgroup='2',
            ), row=2, col=1
        )
        # Slow signal (%d)
        fig.append_trace(go.Scatter(
                x=data.index,
                y=data['macd_signal'],
                line=dict(color='#6495ed', width=2),
                #showlegend=False,
                legendgroup='2',
                name='MACD signal'
            ), row=2, col=1
        )
        # Colorize the histogram values
        colors = np.where(data['macd_CD'] < 0, '#ff6347', '#00ff00')
        # Plot the histogram
        fig.append_trace(
            go.Bar(
                x=data.index,
                y=data['macd_CD'],
                name='Histogram',
                marker_color=colors,
            ), row=2, col=1
        )
        # Make it pretty
        layout = go.Layout(
            plot_bgcolor='#f5f5f5',
            # Font Families
            font_family='Balto',
            font_color='#000000',
            font_size=20,
            xaxis=dict(
                rangeslider=dict(
                    visible=False
                )
            )
        )
        # Update options and show plot
        fig.update_layout(layout)
        return fig




    # dealing with Different Timeframe, assign a resolution to each time frame
    # outputs a plotly express plot

    if time_frame == '24H':
        # the number inside tail() is (how many min in 24h) so --> (number of 'resolution' in 'time_frame')
        graph_data = process_data(data,resolution='min').tail(1440)
        figure = make_graphs(graph_data)
        return figure

    if time_frame == 'ALL':
        graph_data = process_data(data,resolution='D')
        figure = make_graphs(graph_data)
        return figure

    if time_frame == '1W':
        graph_data = process_data(data,resolution='15Min').tail(672)
        figure = make_graphs(graph_data)
        return figure

    if time_frame == '1M':
        graph_data = process_data(data,resolution='3H').tail(240)
        figure = make_graphs(graph_data)  
        return figure

    if time_frame == '6M':
        graph_data = process_data(data,resolution='D').tail(180)
        figure = make_graphs(graph_data)
        return figure

    if time_frame == '1Y':
        graph_data = process_data(data,resolution='D').tail(365)
        figure = make_graphs(graph_data)
        return figure       

    # if something not included gets put iin, just graph the all time data 
        # without any indicators
    else:
        graph_data = process_data(data = data, resolution = 'D')
        figure = px.line(data_frame=graph_data, y = ['close'] )
        layout = go.Layout(
            plot_bgcolor='#f5f5f5',
            # Font Families
            font_family='Balto',
            font_color='#000000',
            font_size=18,
            xaxis=dict(
                rangeslider=dict(
                    visible=False
                )
            )
        )
        # Update options and show plot
        figure.update_layout(layout)
        return figure    







## graph_candlestick function

In [4]:
# this is the dedicated function for MACD analysis
# TO USE THE FUNCTION, RUN THIS BLOCK
def graph_candlestick(data,time_frame = 'ALL', sma_rate_1 = 20, sma_rate_2 = 50 ):
    import pandas as pd
    import numpy as np
    import plotly.express as px
    from datetime import datetime, date
    import plotly.graph_objects as go

 
    

    # This function converts the timestamp into pandas datetime and set it as index
    def change_date(df):
        x = [i for i in range(len(df.columns.isin(['time','open','close','high','low','volume'])-1)) if df.columns.isin(['time','open','close','high','low','volume'])[i] == False ]
        df.drop(columns=df.columns[x],inplace =True)
        if 'time' in list(df.columns):
            df['time']= pd.to_datetime(df['time'],unit = 'ms')
            df = df.set_index('time',drop = True)
        df.sort_index
        return df


    # this function resamples the data into sepcified resolution
    def resample(data,resolution ='D'):
        data = data.resample(resolution).mean()
        data = data.dropna()
        return data
    
    # this function calculates SMA based on given rate
    def get_sma(prices, rate= 20): 
        return prices.rolling(rate).mean()



    # this function will return a DataFrame with specified parameters,
    # later the data produced will be used to graph
    # The "resolution" parameter will be determined based on "time_frame"
    def process_data(data,resolution = 'D', sma_rate_1 = 20, sma_rate_2 = 50):
        # first convert the data
        data = change_date(data)

        #then resample the data
        # if the resolution is not already "Min"
        if resolution != 'Min':
            data = resample(data,resolution)

        # calculate the SMA with different periods
        data['sma_1'] = get_sma(data.close,rate = sma_rate_1)
        data['sma_2'] = get_sma(data.close,rate = sma_rate_2)
        

        # return the DataFrame with all data processed, and Includes all Tech Indicators calculated based on
        # specified resolution 
        return data



    def make_graphs(data,sma_rate_1 = sma_rate_1, sma_rate_2 = sma_rate_2):
        import numpy as np
        import plotly.express as px
        from datetime import datetime, date
        import plotly.graph_objects as go
        
        # create the graph
        fig = go.Figure()

        fig.add_trace(go.Candlestick(x=data.index,
                        open=data['open'],
                        high=data['high'],
                        low=data['low'],
                        close=data['close'],
                        showlegend=False
                                    )
        )
        fig.add_trace(
            {'x': data.index,
            'y': data.sma_1,
            'type': 'scatter',
            'mode': 'lines',
            'line': {
                'width': 1,
                'color': '#6495ed'
                    },
            'name': f'SMA of {sma_rate_1} periods'   
            }

        )

        fig.add_trace(
            {'x': data.index,
            'y': data.sma_2,
            'type': 'scatter',
            'mode': 'lines',
            'line': {
                'width': 1,
                'color': 'blueviolet'
                    },
            'name': f'SMA of {sma_rate_2} periods'   
            }

        )

       
        # Make it pretty
        layout = go.Layout(
            plot_bgcolor='#f5f5f5',
            # Font Families
            font_family='Balto',
            font_color='#000000',
            font_size=13,
            xaxis=dict(
                rangeslider=dict(
                    visible=True
                )
            )
        )
        # Update options and show plot
        fig.update_layout(layout)
        return fig




    # dealing with Different Timeframe, assign a resolution to each time frame


    if time_frame == '24H':
        # the number inside tail() is (how many min in 24h) so --> (number of 'resolution' in 'time_frame')
        graph_data = process_data(data,resolution='Min',sma_rate_1 = sma_rate_1, sma_rate_2 = sma_rate_2).tail(1440)
        figure = make_graphs(graph_data)
        return figure

    if time_frame == 'ALL':
        graph_data = process_data(data,resolution='D',sma_rate_1 = sma_rate_1, sma_rate_2 = sma_rate_2)
        figure = make_graphs(graph_data)
        return figure

    if time_frame == '1W':
        graph_data = process_data(data,resolution='15Min',sma_rate_1 = sma_rate_1, sma_rate_2 = sma_rate_2).tail(672)
        figure = make_graphs(graph_data)
        return figure

    if time_frame == '1M':
        graph_data = process_data(data,resolution='3H',sma_rate_1 = sma_rate_1, sma_rate_2 = sma_rate_2).tail(240)
        figure = make_graphs(graph_data)  
        return figure

    if time_frame == '6M':
        graph_data = process_data(data,resolution='D',sma_rate_1 = sma_rate_1, sma_rate_2 = sma_rate_2).tail(180)
        figure = make_graphs(graph_data)
        return figure

    if time_frame == '1Y':
        graph_data = process_data(data,resolution='D',sma_rate_1 = sma_rate_1, sma_rate_2 = sma_rate_2).tail(365)
        figure = make_graphs(graph_data)
        return figure       

    # if something not included gets put iin, just graph the all time data 
        # without any indicators
    else:
        graph_data = process_data(data = data, resolution = 'D')
        figure = px.line(data_frame=graph_data, y = ['close'] )
        layout = go.Layout(
            plot_bgcolor='#f5f5f5',
            # Font Families
            font_family='Balto',
            font_color='#000000',
            font_size=13,
            xaxis=dict(
                rangeslider=dict(
                    visible=False
                )
            )
        )
        # Update options and show plot
        figure.update_layout(layout)
        return figure    







## this new function calculate the return given the period (ROI)

In [5]:
# this is the dedicated function for MACD analysis
# TO USE THE FUNCTION, RUN THIS BLOCK
def calculate_return(data,time_frame = 'ALL'):
    import pandas as pd
    import numpy as np

    from datetime import datetime, date


 
    

    # This function converts the timestamp into pandas datetime and set it as index
    def change_date(df):
        x = [i for i in range(len(df.columns.isin(['time','open','close','high','low','volume'])-1)) if df.columns.isin(['time','open','close','high','low','volume'])[i] == False ]
        df.drop(columns=df.columns[x],inplace =True)
        if 'time' in list(df.columns):
            df['time']= pd.to_datetime(df['time'],unit = 'ms')
            df = df.set_index('time',drop = True)
        df.sort_index
        return df


    # this function resamples the data into sepcified resolution
    def resample(data,resolution ='D'):
        data = data.resample(resolution).mean()
        data = data.dropna()
        return data

    
    # this function will return a DataFrame with specified parameters,
    # later the data produced will be used to graph
    # The "resolution" parameter will be determined based on "time_frame"
    def process_data(data,resolution = 'D'):
        # first convert the data
        data = change_date(data)

        #then resample the data
        # if the resolution is not already "Min"
        if resolution != 'Min':
            data = resample(data,resolution)


        # return the DataFrame with all data processed, and Includes all Tech Indicators calculated based on
        # specified resolution 
        return data
    
    
    # function to calculate the return rate
    
    def calculate(begin, end):
        percent_change = ((end-begin)/begin)*100
        return f"{round(percent_change[0],2)}%"


    # dealing with Different Timeframe, assign a resolution to each time frame


    if time_frame == '24H':
        # the number inside tail() is (how many min in 24h) so --> (number of 'resolution' in 'time_frame')
        selected_data = process_data(data,resolution='Min').tail(1440)
        num = calculate(begin = selected_data.head(1).close.values, end = selected_data.tail(1).close.values)
        return num

    if time_frame == 'ALL':
        selected_data = process_data(data,resolution='D')
        num = calculate(begin = selected_data.head(1).close.values, end = selected_data.tail(1).close.values)
        return num


    if time_frame == '1W':
        selected_data = process_data(data,resolution='Min').tail(672)
        num = calculate(begin = selected_data.head(1).close.values, end = selected_data.tail(1).close.values)
        return num

    if time_frame == '1M':
        selected_data = process_data(data,resolution='3H').tail(240)
        num = calculate(begin = selected_data.head(1).close.values, end = selected_data.tail(1).close.values)
        return num

    if time_frame == '6M':
        selected_data = process_data(data,resolution='D').tail(180)
        num = calculate(begin = selected_data.head(1).close.values, end = selected_data.tail(1).close.values)
        return num

    if time_frame == '1Y':

        selected_data = process_data(data,resolution='D').tail(365)
        num = calculate(begin = selected_data.head(1).close.values, end = selected_data.tail(1).close.values)
        return num

    # if something not included gets put iin, just graph the all time data 
        # without any indicators







In [6]:
btcusd = read_data('btcusd')

In [7]:
calculate_return(btcusd, 'ALL')

'42518.61%'

In [8]:
#graph_candlestick(btcusd,'ALL')

In [10]:

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np

def read_data(name = 'btcusd'):
    import pandas as pd
    path = f"data/{name}.csv"
    df =pd.read_csv(path)    
    return df



# The next box of markdown was the code for my page, I've incooperated it into the frame within the bigger framework 

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

controls = dbc.Card(
    [
        html.Div(
            [
                dbc.Label("Crypto Currency"),
                dcc.Dropdown(
                    id="chosen_currency_candlestick",
                    options=[
                       {'label':c, 'value':c} for c in ['BTCUSD', 'ETHUSD','SOLUSD']
                    ],
                    value="BTCUSD",
                    clearable = False,
                ),
            ]
        ),
        html.Div(
            [
                dbc.Label("Timeframe shown:"),
                dcc.Dropdown(
                    id="time_frame_candlestick",
                    options=[
                        {'label':c,'value':c} for c in ['ALL','1Y','6M','1M','1W','24H']
                    ],
                    value="24H",
                    clearable = False,
                ),
            ]
        ),

    ],
    body=True,
)

app.layout = dbc.Container(
    [
        html.H1("Quick look Candlestick graph"),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(controls, md=4),
                dbc.Col(dcc.Graph(id="candlestick")),
#                dbc.Col(html.Label(id = 'return_candle'))
                dbc.Row(html.Div(id= 'table_container')),
            ],
            align="center",
        ),
        
    ],
    fluid=True,
    
)


## call backs go here


@app.callback(   # this callback updates the graph
    Output('candlestick','figure'),
#    Output('return_candle','children'),
    [Input("chosen_currency_candlestick",'value'),
     Input("time_frame_candlestick",'value')]
)
# the callback for graph_candlestick graph
def update_candlestick(chosen_currency_candlestick, time_frame_candlestick):
    df = read_data(chosen_currency_candlestick.lower())
    time_frame = time_frame_candlestick
    figure = graph_candlestick(df,time_frame)
    
#    num = calculate_return(chosen_currency_candlestick, time_frame_candlestick)
    return figure

@app.callback(     # this callback updates the table of contents(return for the candlestick page)
    Output('table_container','children'),
    [Input('chosen_currency_candlestick','value'),
     Input('time_frame_candlestick','value')]
)

def update_table(chosen_currency_candlestick, time_frame_candlestick):
    df = read_data(chosen_currency_candlestick.lower())
    percent_change = calculate_return(df, time_frame_candlestick)
    df = pd.DataFrame(
        {
            'content':['Return within period:'],
            'value':[percent_change]
        }
    )
    table = dbc.Table.from_dataframe(df,striped=True, bordered=True, hover=True)
    return table
    

if __name__ == "__main__":
    app.run_server(port=8910)


# Ishan - throwing in code for the header and attempting to add it in

In [15]:
#price displaying divs. right now ive set the price to just 1

btcd=html.Div(children = html.Div('1',style={'font-size':'20px','font-family':'sans-serif'}), style={'color': 'white', 'backgroundColor': '#cfc16b'})
ethd=html.Div(children = html.Div('1',style={'font-size':'20px','font-family':'sans-serif'}), style={'color': 'white', 'backgroundColor': '#cfc16b'})
bnbd=html.Div(children = html.Div('1',style={'font-size':'20px','font-family':'sans-serif'}), style={'color': 'white', 'backgroundColor': '#cfc16b'})
xrpd=html.Div(children = html.Div('1',style={'font-size':'20px','font-family':'sans-serif'}), style={'color': 'white', 'backgroundColor': '#cfc16b'})
adad=html.Div(children = html.Div('1',style={'font-size':'20px','font-family':'sans-serif'}), style={'color': 'white', 'backgroundColor': '#cfc16b'})
sold=html.Div(children = html.Div('1',style={'font-size':'20px','font-family':'sans-serif'}), style={'color': 'white', 'backgroundColor': '#cfc16b'})
terrad=html.Div(children = html.Div('1',style={'font-size':'20px','font-family':'sans-serif'}), style={'color': 'white', 'backgroundColor': '#cfc16b'})
avaxd=html.Div(children = html.Div('1',style={'font-size':'20px','font-family':'sans-serif'}), style={'color': 'white', 'backgroundColor': '#cfc16b'})

# The code for the dashboard

In [ ]:
app = dash.Dash(external_stylesheets=[dbc.themes.YETI])

# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "19rem",
    "padding": "2rem 1rem",
    "background-color": "#6baaa6",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "19rem",
    "margin-right": "1rem",
    "padding": "2rem 1rem",
#    "background-color": "#26609f"
}

controls = dbc.Card(  # this is the control panel of the main page (candlestick)
    [
        html.Div(
            [
                dbc.Label("Crypto Currency"),
                dcc.Dropdown(
                    id="chosen_currency_candlestick",
                    options=[
                       {'label':c, 'value':c} for c in ['BTCUSD', 'ETHUSD','SOLUSD']
                    ],
                    value="BTCUSD",
                    clearable = False,
                ),
            ]
        ),
        html.Div(
            [
                dbc.Label("Timeframe shown:"),
                dcc.Dropdown(
                    id="time_frame_candlestick",
                    options=[
                        {'label':c,'value':c} for c in ['ALL','1Y','6M','1M','1W','24H']
                    ],
                    value="24H",
                    clearable = False,
                ),
            ]
        ),

    ],
    body=True,
)


sidebar = html.Div(
    [
        html.P("Crypto Dashboard", className="display-4"),
        html.Hr(),
        html.P(
            "A cryptocurrency asset dashboard for Differential Capital", className="lead"
        ),
        dbc.Nav(
            [
                dbc.NavLink("Home", href="/", active="exact"),
                dbc.NavLink("Stats", href="/stats", active="exact"),
                dbc.NavLink("Sentiment Analysis", href="/sentiment", active="exact"),
                dbc.NavLink("Charting", href="/charting", active="exact")
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id="page-content", style=CONTENT_STYLE)

app.layout = html.Div([dcc.Location(id="url"), sidebar, content])


@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):
    if pathname == "/":
        return dbc.Container(
    [
        #Ishan adding in code-starting here
        html.Div(style={'backgroundColor': colors['background'], 'height':'125px'}
    
        ,children=[
            html.Div(children=[html.Div('BTC', style={'color':'white','margin-top':'25px','margin-bottom':'25px','backgroundColor': '#297353', 'font-size':'20px','font-family':'sans-serif'}), btcd], style={'width': '11%', 'display': 'inline-block', 'text-align':'center','margin-right': '10px', 'margin-left':'25px'}),
            html.Div(children=[html.Div('ETH', style={'color':'white','margin-top':'25px','margin-bottom':'25px','backgroundColor': '#297353', 'font-size':'20px','font-family':'sans-serif'}), ethd], style={'width': '11%', 'display': 'inline-block', 'text-align':'center','margin-right': '10px'}),
            html.Div(children=[html.Div('BNB', style={'color':'white','margin-top':'25px','margin-bottom':'25px','backgroundColor': '#297353', 'font-size':'20px','font-family':'sans-serif'}), bnbd], style={'width': '11%', 'display': 'inline-block', 'text-align':'center','margin-right': '10px'}),
            html.Div(children=[html.Div('XRP', style={'color':'white','margin-top':'25px','margin-bottom':'25px','backgroundColor': '#297353', 'font-size':'20px','font-family':'sans-serif'}), xrpd], style={'width': '11%', 'display': 'inline-block', 'text-align':'center','margin-right': '10px'}),
            html.Div(children=[html.Div('ADA', style={'color':'white','margin-top':'25px','margin-bottom':'25px','backgroundColor': '#297353', 'font-size':'20px','font-family':'sans-serif'}), adad], style={'width': '11%', 'display': 'inline-block', 'text-align':'center','margin-right': '10px'}),
            html.Div(children=[html.Div('SOL', style={'color':'white','margin-top':'25px','margin-bottom':'25px','backgroundColor': '#297353', 'font-size':'20px','font-family':'sans-serif'}), sold], style={'width': '11%', 'display': 'inline-block', 'text-align':'center','margin-right': '10px'}),
            html.Div(children=[html.Div('TERRA', style={'color':'white','margin-top':'25px','margin-bottom':'25px','backgroundColor': '#297353', 'font-size':'20px','font-family':'sans-serif'}), terrad], style={'width': '11%', 'display': 'inline-block', 'text-align':'center','margin-right': '10px'}),
            html.Div(children=[html.Div('AVAX', style={'color':'white','margin-top':'25px','margin-bottom':'25px','backgroundColor': '#297353', 'font-size':'20px','font-family':'sans-serif'}), avaxd], style={'width': '11%', 'display': 'inline-block', 'text-align':'center','margin-right': '10px'})
        ]),
        #End of Ishan code
        
        
        html.H1("Quick look Candlestick graph"),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(controls, md=4),
                dbc.Col(dcc.Graph(id="candlestick")),
#                dbc.Col(html.Label(id = 'return_candle'))
                dbc.Row(html.Div(id= 'table_container')),
            ],
            align="center",
        ),
        
    ],
    fluid=True,)
    elif pathname == "/stats":
        return html.P("Stats about various assets would go here!")
    elif pathname == "/sentiment":
        return html.P("Sentiment Analysis would go here!")
    elif pathname == "/charting":
        return html.P("Technical Analysis and Charting would go here!")
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )


@app.callback(   # this callback updates the candlestick graph
    Output('candlestick','figure'),
#    Output('return_candle','children'),
    [Input("chosen_currency_candlestick",'value'),
     Input("time_frame_candlestick",'value')]
)
# the function for graph_candlestick graph
def update_candlestick(chosen_currency_candlestick, time_frame_candlestick):
    df = read_data(chosen_currency_candlestick.lower())
    time_frame = time_frame_candlestick
    figure = graph_candlestick(df,time_frame)
    
#    num = calculate_return(chosen_currency_candlestick, time_frame_candlestick)
    return figure

@app.callback(     # this callback updates the table of contents(return for the candlestick page)
    Output('table_container','children'),
    [Input('chosen_currency_candlestick','value'),
     Input('time_frame_candlestick','value')]
)

def update_table(chosen_currency_candlestick, time_frame_candlestick):  # creae a dataframe with contents such as return rate
    df = read_data(chosen_currency_candlestick.lower())
    percent_change = calculate_return(df, time_frame_candlestick)
    df = pd.DataFrame(
        {
            'content':['Return within period:'],
            'value':[percent_change]
        }
    )
    table = dbc.Table.from_dataframe(df,striped=True, bordered=True, hover=True)  # the dash-boostrap-component conviently converts the data frame into heml table
    return table
    




if __name__ == "__main__":
    app.run_server(port=8910)

In [ ]:
app._terminate_server(port=8910)